In [1]:
# Pip installs
!pip install transformers torch
!pip install transformers[torch]
!pip install accelerate -U

In [2]:
label2id = {"O": 0, "B-CONDITION": 1, "I-CONDITION": 2, "B-MEDICATION": 3, "I-MEDICATION": 4}
id2label = {0: "O", 1: "B-CONDITION", 2: "I-CONDITION", 3: "B-MEDICATION", 4: "I-MEDICATION"}

In [3]:
import torch
from transformers import AutoModelForTokenClassification, AutoTokenizer
import os

# Setting path
base_path = "D:/Abdullah/Documents/MediMatch"
model_relative_path = os.path.join("Abdul/NLP_Models/Medical")
model_path = os.path.join(base_path, model_relative_path)

# Load model and tokenizers
model = AutoModelForTokenClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)


c:\Users\Abdullah\anaconda3\Lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
c:\Users\Abdullah\anaconda3\Lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [4]:
# Example text here
text = "Using Escitalopram for anxiety management"

inputs = tokenizer(text, return_tensors="pt")

with torch.no_grad():
    outputs = model(**inputs)

logits = outputs.logits
predictions = logits.argmax(-1).squeeze().tolist()

predicted_labels = [id2label[label] for label in predictions]

tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"].squeeze().tolist())
labeled_tokens = list(zip(tokens, predicted_labels))

def aggregate_entities(tokens, predicted_labels):
    entities = []
    current_entity = []
    current_label = None

    for token, label in zip(tokens, predicted_labels):
        # Skip over the special tokens
        if token in ["[CLS]", "[SEP]"]:
            continue

        # Subword tokens
        if token.startswith("##"):
            if current_entity:
                current_entity[-1] += token[2:]  # Merge with the previous
            continue

        # BIO tags
        if label.startswith("B-"):
            if current_entity:  # Save the previous 
                entities.append((current_label, " ".join(current_entity)))
            current_entity = [token]
            current_label = label[2:]  # Remove the BIO prefix
        elif label.startswith("I-") and current_label == label[2:]:
            current_entity.append(token)
        else:  # O or different entity
            if current_entity:  # Save the previous 
                entities.append((current_label, " ".join(current_entity)))
                current_entity = []
                current_label = None

    if current_entity:
        entities.append((current_label, " ".join(current_entity)))

    return entities

entities = aggregate_entities(tokens, predicted_labels)

def filter_entities(entities, filter_words):
    filtered_entities = []
    for label, entity in entities:
        filtered_entity_words = [word for word in entity.split() if word.lower() not in filter_words]
        if filtered_entity_words: 
            filtered_entities.append((label, " ".join(filtered_entity_words)))
    return filtered_entities

filter_words = {"on", "for", "currently", "taking"}
filtered_entities = filter_entities(entities, filter_words)

# Print Outptu
for label, entity in filtered_entities:
    print(f"{label}: {entity}")




MEDICATION: Escitalopram
CONDITION: anxiety


In [10]:
def process_text(text):
    inputs = tokenizer(text, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    predictions = logits.argmax(-1).squeeze().tolist()
    predicted_labels = [id2label[label] for label in predictions]
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"].squeeze().tolist())
    labeled_tokens = list(zip(tokens, predicted_labels))

    def aggregate_entities(tokens, predicted_labels):
        entities = []
        current_entity = []
        current_label = None
        for token, label in zip(tokens, predicted_labels):
            if token in ["[CLS]", "[SEP]"]:
                continue
            if token.startswith("##"):
                if current_entity:
                    current_entity[-1] += token[2:]
                continue
            if label.startswith("B-"):
                if current_entity:
                    entities.append((current_label, " ".join(current_entity)))
                current_entity = [token]
                current_label = label[2:]
            elif label.startswith("I-") and current_label == label[2:]:
                current_entity.append(token)
            else:
                if current_entity:
                    entities.append((current_label, " ".join(current_entity)))
                    current_entity = []
                    current_label = None
        if current_entity:
            entities.append((current_label, " ".join(current_entity)))
        return entities

    entities = aggregate_entities(tokens, predicted_labels)

    def filter_entities(entities, filter_words):
        filtered_entities = []
        for label, entity in entities:
            filtered_entity_words = [word for word in entity.split() if word.lower() not in filter_words]
            if filtered_entity_words:
                filtered_entities.append((label, " ".join(filtered_entity_words)))
        return filtered_entities

    filter_words = {"of", "from", "to", "and"}
    filtered_entities = filter_entities(entities, filter_words)
    return filtered_entities

# Test data geneerated from CHATGPT
test_data = [
    "Prescribed Zyrtec for migraines."
    "Suffering from insomnia."
    "Using Escitalopram for anxiety management"
    "Diagnosed with osteoarthritis."
    "Mother is on Haldol for schizophrenia."
    "Diagnosed with fibromyalgia."
    "Currently on Lisinopril for hypertension."
    "Currently taking Effexor for migraines."
    "Experiencing insomnia."
    "Using Ibuprofen for anxiety."
    "Diagnosed with endometriosis."
    "Taking Lorazepam for anxiety."
    "Suffering from migraines."
    "Diagnosed with ulcerative colitis."
    "Using Lexapro."
    "Suffering from seizures."
    "Suffering from migraines."
    "Using Haldol."
    
]

# # Processing each phrase
for text in test_data:
    filtered_entities = process_text(text)
    for label, entity in filtered_entities:
        print(f"{label}: {entity}")
    print("------")

MEDICATION: Zyrtec
CONDITION: insomnia
MEDICATION: Escitalopram
CONDITION: for anxiety
CONDITION: with osteoarthritis
MEDICATION: on Haldol
CONDITION: with fibromyalgia
MEDICATION: on Lisinopril
MEDICATION: taking Effexor
CONDITION: insomnia
CONDITION: with endometriosis
CONDITION: migraines
CONDITION: with ulcerative colitis
MEDICATION: Using Lexapro
CONDITION: seizures
CONDITION: migraines
------


In [6]:
def process_text(text):
    inputs = tokenizer(text, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    predictions = logits.argmax(-1).squeeze().tolist()
    predicted_labels = [id2label[label] for label in predictions]
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"].squeeze().tolist())
    labeled_tokens = list(zip(tokens, predicted_labels))

    def aggregate_entities(tokens, predicted_labels):
        entities = []
        current_entity = []
        current_label = None
        for token, label in zip(tokens, predicted_labels):
            if token in ["[CLS]", "[SEP]"]:
                continue
            if token.startswith("##"):
                if current_entity:
                    current_entity[-1] += token[2:]
                continue
            if label.startswith("B-"):
                if current_entity:
                    entities.append((current_label, " ".join(current_entity)))
                current_entity = [token]
                current_label = label[2:]
            elif label.startswith("I-") and current_label == label[2:]:
                current_entity.append(token)
            else:
                if current_entity:
                    entities.append((current_label, " ".join(current_entity)))
                    current_entity = []
                    current_label = None
        if current_entity:
            entities.append((current_label, " ".join(current_entity)))
        return entities

    entities = aggregate_entities(tokens, predicted_labels)

    def filter_entities(entities, filter_words):
        filtered_entities = []
        for label, entity in entities:
            filtered_entity_words = [word for word in entity.split() if word.lower() not in filter_words]
            if filtered_entity_words:
                filtered_entities.append((label, " ".join(filtered_entity_words)))
        return filtered_entities

    filter_words = {"of", "from", "to", "and"}
    filtered_entities = filter_entities(entities, filter_words)
    return filtered_entities

# Test data geneerated from CHATGPT
test_data = [
"I find joy in cooking healthy meals with fresh ingredients."
"I prioritize regular physical activity to maintain my fitness levels."
"I make time for hobbies that bring me joy and reduce stress."
"I prioritize self-care activities like baths and massages for relaxation."
"I enjoy family picnics as a way to bond and enjoy nutritious meals together."
"I prioritize sleep hygiene practices for better sleep and overall health."
"I make time for laughter and humor to improve my mood and well-being."
"I prioritize healthy snacks like fruits and nuts for sustained energy."
"I find joy in spending time with pets for my mental and emotional health."
"I enjoy swimming as a refreshing and low-impact form of exercise."
"I prioritize relaxation techniques like meditation for stress relief."
"I make time for regular family walks to catch up and connect."
"I prioritize regular check-ups to monitor my health and well-being."
"I find joy in exploring nature through activities like birdwatching."
"I prioritize spending time in nature to recharge and rejuvenate."
"I enjoy dancing to my favorite music as a form of exercise and self-expression."
"I prioritize spending time with loved ones to maintain strong relationships."
"I make time for hobbies like painting or crafting to relax and unwind."
"I enjoy cooking healthy meals as a way to nourish my body and soul."
"I prioritize time for relaxation and self-care to reduce stress levels."
"I practice mindfulness and meditation to stay grounded and focused."
"I prioritize getting enough sleep to ensure I have the energy for the day."
"I enjoy reading books to expand my knowledge and escape into different worlds."
"I volunteer at local charities to give back to the community and help those in need."
"I engage in regular physical activity to maintain my overall health and well-being."
]

# Processing each phrase
for text in test_data:
    filtered_entities = process_text(text)
    for label, entity in filtered_entities:
        print(f"{label}: {entity}")
    print("------")

------


In [1]:
# Values
Total = 50
True_Positives = 19
False_Positives = 6
total_empty = 25
True_Negatives = 25
False_Negatives = 0

# Calculate 
accuracy = (True_Positives + True_Negatives) / Total
precision = True_Positives / (True_Positives + False_Positives)
recall = True_Positives / (True_Positives + False_Negatives)
specificity = True_Negatives / (True_Negatives + False_Positives)
f1_score = 2 * (precision * recall) / (precision + recall)

print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"Specificity: {specificity:.2f}")
print(f"F1 Score: {f1_score:.2f}")


Accuracy: 0.88
Precision: 0.76
Recall: 1.00
Specificity: 0.81
F1 Score: 0.86


In [ ]:
# ACROSS ALL 3 MODELS:
# Average Accuracy: 0.91
# Average Precision: 0.8033
# Average Recall: 0.9733
# Average Specificity: 0.7733
# Average F1 Score: 0.88